**IMPORTANTE**
L'elaborazione è stata eseguita utilizzando Google Colab

Installo alcune librerie necessarie all'elaborazione dati

In [90]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [91]:
pip install pyforest

In [92]:
pip install spacy

In [93]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [94]:
import pyforest

In [95]:
file = '/content/drive/My Drive/ColabNotebooks/AnalisiNLP/train_set.csv'

In [96]:
df = pd.read_csv(file)
df

<IPython.core.display.Javascript object>

,Job_offer,Label
0,"Openjobmetis SpA ricerca, per importante azien...",Java Developer
1,"La persona prescelta, diplomata o laureata in ...",Software Engineer
2,Sei un informatico o matematico con la passion...,Software Engineer
3,"Ti occuperai della progettazione, realizzazion...",Programmer
4,Stiamo cercando uno\una sviluppatore\sviluppat...,Programmer
...,...,...
1747,"* Test JUnit\n * Git, La Ibs Srl è alla ri...",Java Developer
1748,"La ricorsa, inserita all'interno di un team, l...",Programmer
1749,Sviluppatore Java Junior da inserire in attivi...,Java Developer
1750,La risorsa sarà inserita nei team di risorse d...,Web Developer


Tronco il testo a massimo 500 caratteri per Job_offer in quanto ho deciso di tradurlo in inglese e i traduttori a disposizione non consentono un numero superiore di caratteri da tradurre.
Infine ricostruisco il dataset.

In [97]:
def smart_truncate(content, length= 500, suffix='...'):
  if len(content) <= length:
    return content
  else:
    return ' '.join(content[:length+1].split(' ')[0:-1]) + suffix

In [98]:
x = []
 
for i, row_value in df['Job_offer'].iteritems():
  a = smart_truncate(df['Job_offer'][i])
  x.append(a)

In [99]:
y = []
 
for i, row_value in df['Label'].iteritems():
  y.append(row_value)

In [100]:
df = pd.DataFrame({'Job_offer': x, 'Label': y})

<IPython.core.display.Javascript object>

In [101]:
#Uno sguardo al dataset

print(df['Job_offer'][1749])
 
print(' ')
 
print(len(df.Label.unique()))
 
print(' ')
 
print(df.Label.unique())

Sviluppatore Java Junior da inserire in attività progettuali.
 
5
 
['Java Developer' 'Software Engineer' 'Programmer' 'System Analyst'
 'Web Developer']


<p>
    <font color="red"><strong> PRELABORAZIONE DEL TESTO </strong></font>
</p>

In [102]:
pip install deep-translator

In [103]:
from deep_translator import GoogleTranslator

In [ ]:
#Traduco il testo in Job_offer in inglese come precedentemente indicato

translator = GoogleTranslator()
 
df['English'] = df['Job_offer'].apply(translator.translate, src='it', dest='en')

In [ ]:
import spacy
import en_core_web_sm

In [ ]:
nlp = en_core_web_sm.load()

In [ ]:
#Lemmatizza e elimina stopwords
 
# Nuova lista di stopwords
customize_stop_words = [
    'and',
    'or',
    ',',
    ':',
    '!',
    '\n',
    '*',
    '/'
        
]
 
 
# Indichiamo che sono stopwords
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True
    
 
# Converti ogni riga nel documento di rappresentanza e restituire il lemma dei token
# Il token se rappresenta una stopwords o un segno di punteggiatura non viene considerato. Finalmente unisci i lemmi in una stringa

X = df.English.apply(lambda text: 
                    " ".join(token.lemma_ for token in nlp(text) 
                             if not token.is_stop and not token.is_punct))


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#Definisco numero massimo di parole da prendere in considerazione tra quelle piu comuni
#quindi il vocabolario di testo che vado a caricare sara costituito solo dalle 10 mila parole piu comuni
max_words = 8000
 
#Definisco lunghezza massima di parole (token) di una Job offer
maxlen = 8000

In [ ]:
#Tokenizzo le parole all interno di ogni Job Offer tradotta
 
tokenizer = Tokenizer(num_words = max_words)
 
tokenizer.fit_on_texts(X)
 
sequences = tokenizer.texts_to_sequences(X)

In [ ]:
word_index = tokenizer.word_index

In [ ]:
print('Found %s unique tokens.' % len(word_index))

In [ ]:
#Consente di definire numero massimo parole di ogni job offer e inserisce il padding ovvero gli zeri per rendere
#le frasi della stessa lunghezza qualora siano piu brevi del numero di parole massime prestabilite
 
data = pad_sequences(sequences, maxlen=maxlen) 
 
print('Shape of data tensor:', data.shape)

In [ ]:
#Preprocessiamo le label
 
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
 
y = np.array(df.Label.tolist())
 
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))
 
yy = pd.DataFrame(yy)
 
yy

<p>
    <font color="red"><strong> ADDESTRAMENTO MODELLO </strong></font></p>

In [ ]:
#Divisione del dataset in parte di train e di test

from sklearn.model_selection import train_test_split
 
x_train, x_test, y_train, y_test = train_test_split(data, yy, test_size=0.2, random_state=42)

In [ ]:
max_features = 8000

In [ ]:
from keras import preprocessing
 
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
#Scarica gli Embedding preaddestrati di Glove che e' addestrato su tutta Wikipedia in inglese. Per questo ho deciso di tradurre le Job_offer in inglese.

!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove.6B.zip

In [ ]:
def load_embedding(filename):
  
  with open(filename) as file:
    lines = file.readlines()
 
  embedding = dict()
  
  for line in lines:
    
    parts = line.split()
    embedding[parts[0]] = np.asarray(parts[1:])
    
  return embedding

In [ ]:
#Carico gli Embedding preaddestrati di Glove

import numpy as np
 
filename = '/content/drive/My Drive/ColabNotebooks/AnalisiNLP/glove.6B.200d.txt'
 
raw_embedding = load_embedding(filename)

In [ ]:
def get_weight_matrix(embedding, word_index):
  
  vocab_size = len(word_index)+3
  
  weight_matrix = np.zeros((vocab_size, 200))
 
  for word, i in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
      weight_matrix[i] = vector
      
  return weight_matrix
 
embedding_vectors = get_weight_matrix(raw_embedding, word_index)
embedding_vectors.shape

(3400, 200)

In [ ]:
#importo librerie necessarie per l'addestramento del modello

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Embedding
from keras import layers
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

In [ ]:
embedding_layer = Embedding(embedding_vectors.shape[0], 200, weights=[embedding_vectors], trainable=False, input_length=maxlen, mask_zero=True)

In [ ]:
#Creo il modello

model = Sequential()

model.add(embedding_layer)

model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(LSTM(100))

model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer = 'adam')

In [ ]:
model.summary()

In [ ]:
#Addestro il modello

from datetime import datetime 
 
num_epochs = 15
num_batch_size = 32
 
start = datetime.now()
 
model.fit(x_train, y_train, batch_size= num_batch_size, epochs= num_epochs, validation_data= (x_test, y_test), verbose=1)
 
duration = datetime.now() - start

print("Training completed in time: ", duration)

Epoch 1/15
44/44 [==============================] - 223s 5s/step - loss: 0.1412 - accuracy: 0.9565 - val_loss: 1.5675 - val_accuracy: 0.6467
Epoch 2/15
44/44 [==============================] - 220s 5s/step - loss: 0.1053 - accuracy: 0.9743 - val_loss: 1.6047 - val_accuracy: 0.6752
Epoch 3/15
44/44 [==============================] - 224s 5s/step - loss: 0.0873 - accuracy: 0.9786 - val_loss: 1.6156 - val_accuracy: 0.6752
Epoch 4/15
44/44 [==============================] - 224s 5s/step - loss: 0.0701 - accuracy: 0.9829 - val_loss: 1.7085 - val_accuracy: 0.6809
Epoch 5/15
44/44 [==============================] - 222s 5s/step - loss: 0.0551 - accuracy: 0.9864 - val_loss: 1.7779 - val_accuracy: 0.7009
Epoch 6/15
44/44 [==============================] - 224s 5s/step - loss: 0.0469 - accuracy: 0.9872 - val_loss: 1.8263 - val_accuracy: 0.6838
Epoch 7/15
44/44 [==============================] - 224s 5s/step - loss: 0.0477 - accuracy: 0.9857 - val_loss: 1.8726 - val_accuracy: 0.7037
Epoch 8/15
44

In [ ]:
model.save('/content/drive/My Drive/ColabNotebooks/AnalisiNLP/Soluzione/modello.h5')

In [ ]:
#Carico il modello preaddestrato e gia disponibile nel file Zip con il nome di modello.h5

from keras.models import load_model
 
model = load_model('/content/drive/My Drive/ColabNotebooks/AnalisiNLP/Soluzione/modello.h5')

<p><font color="red"><strong> ANALISI DEL DATASET DI TEST </strong></font></p>

Eseguo le medesime operazioni di prelaborazione e di analisi utilizzate per la creazione del modello

In [ ]:
test = '/content/drive/My Drive/ColabNotebooks/AnalisiNLP/test_set.csv'

In [ ]:
df2 = pd.read_csv(test)
df2

In [ ]:
def smart_truncate(content, length= 500, suffix='...'):
    if len(content) <= length:
        return content
    else:
        return ' '.join(content[:length+1].split(' ')[0:-1]) + suffix

In [ ]:
x = []
 
for i, row_value in df2['Job_offer'].iteritems():
  a = smart_truncate(df2['Job_offer'][i])
  x.append(a)

In [ ]:
y = []
 
for i, row_value in df2['Label'].iteritems():
  y.append(row_value)

In [ ]:
df2 = pd.DataFrame({'Job_offer': x, 'Label': y})

In [ ]:
translator = GoogleTranslator()
 
df2['English'] = df2['Job_offer'].apply(translator.translate, src='it', dest='en')

In [ ]:
Z = df2.English.apply(lambda text: 
                    " ".join(token.lemma_ for token in nlp(text) 
                             if not token.is_stop and not token.is_punct))

In [ ]:
sequences_test = tokenizer.texts_to_sequences(Z)

In [ ]:
X_test = pad_sequences(sequences_test, maxlen=maxlen) 
 
print('Shape of data tensor:', X_test.shape)

In [ ]:
Y_test = np.array(df2.Label.tolist())
 
Y_test = to_categorical(le.transform(Y_test))
 
Y_test = pd.DataFrame(Y_test)
 
Y_test

In [ ]:
model.evaluate(X_test, Y_test)


<p><font color="red"><strong> PREDIZIONI E CLASSIFICATION REPORT </strong></font></p>


In [ ]:
y_pred = model.predict(X_test)
pred = np.argmax(y_pred, axis = 1)

In [ ]:
Y_test = Y_test.transpose()

In [ ]:
rounded_labels=np.argmax([Y_test], axis=1)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

print(classification_report(rounded_labels[0], pred))